# 📊 Estrutura do Banco de Dados - Jarvis

Este notebook mostra a estrutura completa do banco de dados SQLite do projeto Jarvis.

## Objetivo
- Visualizar todas as tabelas do banco
- Ver a estrutura (schema) de cada tabela
- Consultar dados de exemplo
- Verificar relacionamentos entre tabelas


In [1]:
# Importações necessárias
import sqlite3
from pathlib import Path
import pandas as pd
from IPython.display import display, Markdown

print("✅ Bibliotecas importadas com sucesso!")


✅ Bibliotecas importadas com sucesso!


In [2]:
# Caminho do banco de dados
db_path = Path("../jarvis.db")

if not db_path.exists():
    print("⚠️ Banco de dados não encontrado!")
    print(f"   Procurando em: {db_path.absolute()}")
    print("   Execute o bot pelo menos uma vez para criar o banco.")
else:
    print(f"✅ Banco encontrado: {db_path.absolute()}")
    print(f"   Tamanho: {db_path.stat().st_size / 1024:.2f} KB")


✅ Banco encontrado: /Users/eduardosiqueirabonfim/jarvis-planejador-financeiro/notebooks/../jarvis.db
   Tamanho: 108.00 KB


In [3]:
# Conecta ao banco de dados
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row  # Retorna resultados como dicionários

# Lista todas as tabelas
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
tables = cursor.fetchall()

display(Markdown("## 📋 Tabelas no Banco de Dados"))
print(f"Total de tabelas: {len(tables)}\n")
for table in tables:
    print(f"  • {table[0]}")


## 📋 Tabelas no Banco de Dados

Total de tabelas: 6

  • categories
  • conversation_history
  • sqlite_sequence
  • transactions
  • user_rules
  • users


## 🔍 Estrutura das Tabelas (Schema)


In [4]:
# Função para mostrar estrutura de uma tabela
def show_table_schema(table_name):
    """Mostra a estrutura (CREATE TABLE) de uma tabela."""
    cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    result = cursor.fetchone()
    if result:
        display(Markdown(f"### Tabela: `{table_name}`"))
        print("```sql")
        print(result[0])
        print("```\n")

# Mostra estrutura de todas as tabelas
for table in tables:
    show_table_schema(table[0])


### Tabela: `categories`

```sql
CREATE TABLE categories (
                category_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_phone TEXT,
                category_name TEXT,
                description TEXT,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (user_phone) REFERENCES users(user_phone)
            )
```



### Tabela: `conversation_history`

```sql
CREATE TABLE conversation_history (
                message_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_phone TEXT,
                user_message TEXT,
                bot_response TEXT,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (user_phone) REFERENCES users(user_phone)
            )
```



### Tabela: `sqlite_sequence`

...

```sql
CREATE TABLE transactions (
                transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_phone TEXT,
                category_id INTEGER,
                amount REAL,
                expense_description TEXT,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (user_phone) REFERENCES users(user_phone),
                FOREIGN KEY (category_id) REFERENCES categories(category_id)
            )
```



### Tabela: `user_rules`

```sql
CREATE TABLE user_rules (
                rule_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_phone TEXT,
                category_id INTEGER,
                period_type TEXT DEFAULT 'mensal',
                period_start DATETIME DEFAULT CURRENT_TIMESTAMP,
                period_end DATETIME,
                limit_value REAL,
                current_total REAL DEFAULT 0,
                last_updated DATETIME,
                active INTEGER DEFAULT 1,
                FOREIGN KEY (user_phone) REFERENCES users(user_phone),
                FOREIGN KEY (category_id) REFERENCES categories(category_id)
            )
```



### Tabela: `users`

```sql
CREATE TABLE users (
                user_phone TEXT PRIMARY KEY,
                user_name TEXT,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                last_message_at DATETIME,
                setup_step TEXT
            )
```



## 📊 Dados das Tabelas


In [5]:
# Verifica usuários
display(Markdown("### 👥 Tabela: `users`"))
users_df = pd.read_sql_query("SELECT * FROM users", conn)
print(f"Total de registros: {len(users_df)}")
if len(users_df) > 0:
    display(users_df)
else:
    print("  (Nenhum usuário cadastrado)")


### 👥 Tabela: `users`

Total de registros: 1


,user_phone,user_name,created_at,last_message_at,setup_step
0,1211027367,Eduardo,2025-11-29 20:47:36.740637,2025-11-29 20:47:36.740638,None


In [6]:
# Verifica categorias
display(Markdown("### 📁 Tabela: `categories`"))
categories_df = pd.read_sql_query("SELECT * FROM categories ORDER BY category_name", conn)
print(f"Total de registros: {len(categories_df)}")
if len(categories_df) > 0:
    display(categories_df)
else:
    print("  (Nenhuma categoria cadastrada)")


### 📁 Tabela: `categories`

Total de registros: 10


,category_id,user_phone,category_name,description,created_at
0,1,1211027367,Alimentação,"Mercado, supermercado",2025-11-29 23:47:43
1,7,1211027367,Assinaturas,"Netflix, Spotify, streaming",2025-11-29 23:47:43
2,2,1211027367,Delivery,"iFood, Rappi, pedidos",2025-11-29 23:47:43
3,6,1211027367,Farmácia,"Remédios, medicamentos",2025-11-29 23:47:43
4,8,1211027367,Investimento,"Poupança, ações, aplicações",2025-11-29 23:47:43
5,5,1211027367,Lazer,"Cinema, festas, diversão",2025-11-29 23:47:43
6,4,1211027367,Moradia,"Aluguel, condomínio, contas",2025-11-29 23:47:43
7,10,1211027367,Pets,Categoria: Pets,2025-11-29 23:47:59
8,3,1211027367,Transporte,"Uber, combustível, ônibus",2025-11-29 23:47:43
9,9,1211027367,Viagem,"Passagens, hospedagem, turismo",2025-11-29 23:47:43


In [7]:
# Verifica transações
display(Markdown("### 💰 Tabela: `transactions`"))
transactions_df = pd.read_sql_query("""
    SELECT 
        t.transaction_id,
        t.user_phone,
        c.category_name,
        t.amount,
        t.expense_description,
        t.created_at
    FROM transactions t
    LEFT JOIN categories c ON t.category_id = c.category_id
    ORDER BY t.created_at DESC
    LIMIT 20
""", conn)
print(f"Total de transações (mostrando últimas 20): {len(transactions_df)}")
if len(transactions_df) > 0:
    display(transactions_df)
else:
    print("  (Nenhuma transação encontrada)")


### 💰 Tabela: `transactions`

Total de transações (mostrando últimas 20): 1


,transaction_id,user_phone,category_name,amount,expense_description,created_at
0,1,1211027367,Alimentação,2500.0,compras do mes do mercado,2025-11-29 20:48:51.140243


In [8]:
# Verifica limites (user_rules)
display(Markdown("### 🎯 Tabela: `user_rules` (Limites)"))
limits_df = pd.read_sql_query("""
    SELECT 
        r.rule_id,
        r.user_phone,
        c.category_name,
        r.period_type,
        r.limit_value,
        r.current_total,
        r.active,
        r.last_updated
    FROM user_rules r
    LEFT JOIN categories c ON r.category_id = c.category_id
    ORDER BY r.rule_id
""", conn)
print(f"Total de registros: {len(limits_df)}")
if len(limits_df) > 0:
    display(limits_df)
else:
    print("  (Nenhum limite configurado)")


### 🎯 Tabela: `user_rules` (Limites)

Total de registros: 2


,rule_id,user_phone,category_name,period_type,limit_value,current_total,active,last_updated
0,1,1211027367,Alimentação,mensal,2000.0,2500.0,1,2025-11-29 20:48:51.143982
1,2,1211027367,Transporte,mensal,200.0,0.0,1,2025-11-29 20:48:32.217287


In [9]:
# Verifica histórico de conversas
display(Markdown("### 💬 Tabela: `conversation_history`"))
history_df = pd.read_sql_query("""
    SELECT 
        message_id,
        user_phone,
        SUBSTR(user_message, 1, 50) || '...' as user_message_preview,
        SUBSTR(bot_response, 1, 50) || '...' as bot_response_preview,
        created_at
    FROM conversation_history
    ORDER BY created_at DESC
    LIMIT 20
""", conn)
print(f"Total de mensagens (mostrando últimas 20): {len(history_df)}")
if len(history_df) > 0:
    display(history_df)
else:
    print("  (Nenhuma conversa registrada)")


### 💬 Tabela: `conversation_history`

Total de mensagens (mostrando últimas 20): 8


,message_id,user_phone,user_message_preview,bot_response_preview,created_at
0,8,1211027367,Gastei 2500 reais com as compras do mes do mer...,"✅ Gasto registrado: R$ 2.500,00 em Alimentação...",2025-11-29 20:48:51.146542
1,7,1211027367,Nao...,🎉 *Configuração Concluída!*\n\nTudo pronto par...,2025-11-29 20:48:34.757211
2,6,1211027367,Defina um limite de gastos de transporte de 20...,✅ Limite registrado: *Transporte* = R$ 200.00/...,2025-11-29 20:48:32.220710
3,5,1211027367,Defina um limite de gastos de alimentação de 2...,"✅ Limite registrado: *Alimentação* = R$ 2,000....",2025-11-29 20:48:15.652998
4,4,1211027367,Nao...,✅ *Categorias configuradas!*\n\n💰 *Definir lim...,2025-11-29 20:48:03.738766
5,3,1211027367,Pets...,✅ Categoria *Pets* criada!\n\nQuer adicionar m...,2025-11-29 20:47:59.847069
6,2,1211027367,Eduardo...,"Prazer em te conhecer, *Eduardo*! 👋\n\n✅ *Crie...",2025-11-29 20:47:43.069310
7,1,1211027367,Oi...,🎉 *Olá! Eu sou o Jarvis!*\n\nSeu assistente fi...,2025-11-29 20:47:36.747310


## 📈 Estatísticas Gerais


In [10]:
# Estatísticas gerais do banco
stats = {
    "Total de usuários": len(pd.read_sql_query("SELECT * FROM users", conn)),
    "Total de categorias": len(pd.read_sql_query("SELECT * FROM categories", conn)),
    "Total de transações": len(pd.read_sql_query("SELECT * FROM transactions", conn)),
    "Total de limites configurados": len(pd.read_sql_query("SELECT * FROM user_rules WHERE active = 1", conn)),
    "Total de mensagens no histórico": len(pd.read_sql_query("SELECT * FROM conversation_history", conn))
}

display(Markdown("### 📊 Resumo"))
for key, value in stats.items():
    print(f"  {key}: **{value}**")


### 📊 Resumo

  Total de usuários: **1**
  Total de categorias: **10**
  Total de transações: **1**
  Total de limites configurados: **2**
  Total de mensagens no histórico: **8**


In [11]:
# Estatísticas financeiras (se houver transações)
transactions_total = pd.read_sql_query("SELECT COUNT(*) as total FROM transactions", conn).iloc[0]['total']

if transactions_total > 0:
    display(Markdown("### 💰 Estatísticas Financeiras"))
    
    # Total gasto geral
    total_gasto = pd.read_sql_query("SELECT SUM(amount) as total FROM transactions", conn).iloc[0]['total']
    print(f"Total gasto: R$ {total_gasto:,.2f}" if total_gasto else "Total gasto: R$ 0,00")
    
    # Total por categoria
    gastos_categoria = pd.read_sql_query("""
        SELECT 
            c.category_name,
            COUNT(t.transaction_id) as num_transacoes,
            SUM(t.amount) as total_gasto,
            AVG(t.amount) as media_gasto
        FROM categories c
        LEFT JOIN transactions t ON c.category_id = t.category_id
        GROUP BY c.category_id, c.category_name
        HAVING num_transacoes > 0
        ORDER BY total_gasto DESC
    """, conn)
    
    if len(gastos_categoria) > 0:
        print("\nGastos por categoria:")
        display(gastos_categoria)
    else:
        print("  (Nenhum gasto registrado por categoria)")
else:
    print("  (Nenhuma transação para calcular estatísticas)")


### 💰 Estatísticas Financeiras

Total gasto: R$ 2,500.00

Gastos por categoria:


,category_name,num_transacoes,total_gasto,media_gasto
0,Alimentação,1,2500.0,2500.0


In [12]:
# Verifica relacionamentos e integridade
display(Markdown("### 🔗 Verificação de Integridade"))

# Verifica transações sem categoria
transacoes_sem_categoria = pd.read_sql_query("""
    SELECT COUNT(*) as total 
    FROM transactions 
    WHERE category_id IS NULL
""", conn).iloc[0]['total']
print(f"Transações sem categoria: {transacoes_sem_categoria}")

# Verifica limites sem categoria
limites_sem_categoria = pd.read_sql_query("""
    SELECT COUNT(*) as total 
    FROM user_rules r
    LEFT JOIN categories c ON r.category_id = c.category_id
    WHERE c.category_id IS NULL
""", conn).iloc[0]['total']
print(f"Limites sem categoria válida: {limites_sem_categoria}")

# Verifica categorias órfãs (sem transações e sem limites)
categorias_orfas = pd.read_sql_query("""
    SELECT c.category_id, c.category_name
    FROM categories c
    LEFT JOIN transactions t ON c.category_id = t.category_id
    LEFT JOIN user_rules r ON c.category_id = r.category_id
    WHERE t.transaction_id IS NULL AND r.rule_id IS NULL
""", conn)
print(f"Categorias sem uso (órfãs): {len(categorias_orfas)}")
if len(categorias_orfas) > 0:
    display(categorias_orfas)


### 🔗 Verificação de Integridade

Transações sem categoria: 0
Limites sem categoria válida: 0
Categorias sem uso (órfãs): 8


,category_id,category_name
0,2,Delivery
1,4,Moradia
2,5,Lazer
3,6,Farmácia
4,7,Assinaturas
5,8,Investimento
6,9,Viagem
7,10,Pets


In [13]:
# Fecha a conexão
conn.close()
print("✅ Conexão com o banco de dados fechada.")


✅ Conexão com o banco de dados fechada.
